In [1]:
# pip install mikeio xarray netcdf4

In [30]:
import xarray as xr
import numpy as np

# remove this line if you like more advanced display of xarray datasets
xr.set_options(display_style="text")

In [31]:
from mikeio import Dfs2

In [32]:
dfs = Dfs2("../tests/testdata/gebco_sound.dfs2")
ds = dfs.read()

In [49]:
nx = ds[0].shape[2]
ny = ds[0].shape[1]

In [50]:
x0 = dfs.longitude
y0 = dfs.latitude
x = [x0 + dfs.dx*i for i in range(nx)]
y = [y0 + dfs.dy*i for i in range(ny)]

# Either flip the y coordinates, or flip the data
y = list(reversed(y))

In [51]:
res = {}

spdims = ["lat", "lon"]

if len(ds.time) > 1:
    dims = ["t"] + spdims
    coords = {"t": ds.time}
else:
    dims = spdims
    coords = {}
    
coords["lon"] = xr.DataArray(x, dims="lon", attrs={"standard_name" : "longitude", "units" : "degrees_east"})
coords["lat"] = xr.DataArray(y, dims="lat", attrs={"standard_name" : "latitude", "units" : "degrees_north"})

for item in ds.items:
    v = item.name
    res[v] = xr.DataArray(np.squeeze(ds[v]), dims=dims, 
    attrs={'name': v,
            # TODO add standard name from https://cfconventions.org/standard-names.html
            'units': item.unit.name,
            'eumType' : item.type,
            'eumUnit' : item.unit})        

xr_ds = xr.Dataset(res, coords=coords)


In [52]:
xr_ds

<xarray.Dataset>
Dimensions:    (lat: 264, lon: 216)
Coordinates:
  * lon        (lon) float64 12.2 12.21 12.21 12.21 ... 13.09 13.09 13.09 13.1
  * lat        (lat) float64 56.3 56.29 56.29 56.29 ... 55.21 55.21 55.21 55.2
Data variables:
    Elevation  (lat, lon) float64 -31.0 -30.0 -31.0 -31.0 ... -37.0 -38.0 -38.0

In [54]:
xr_ds.to_netcdf("gebco.nc")

# Clean up

In [11]:
import os
os.remove("gebco.nc")